In [9]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

In [5]:
df = pd.read_csv("SpeedDatingData.csv", encoding="ISO-8859-1")

In [17]:
df.groupby('iid')[['like'].transform(lambda x: (x - x.mean()) / x.std())

0       0.948683
1       0.948683
2       0.948683
3       0.948683
4      -0.948683
          ...   
8373   -2.090029
8374   -1.019526
8375    0.050976
8376   -0.484275
8377   -1.019526
Name: like, Length: 8378, dtype: float64

In [24]:
df['scored_decision'] = df['like'] + df['dec']*0.5

In [26]:
df['stdzed_scored_decision'] = df.groupby('iid')['scored_decision'].transform(lambda x: (x - x.mean()) / x.std())

In [32]:
wave9 = df[df['wave']==9][['iid', 'gender', 'pid', 'stdzed_scored_decision']]

In [59]:
df[df['iid']==209]

,iid,id,gender,idg,condtn,wave,round,position,positin1,order,partner,pid,match,int_corr,samerace,age_o,race_o,pf_o_att,pf_o_sin,pf_o_int,pf_o_fun,pf_o_amb,pf_o_sha,dec_o,attr_o,sinc_o,intel_o,fun_o,amb_o,shar_o,like_o,prob_o,met_o,age,field,field_cd,undergra,mn_sat,tuition,race,imprace,imprelig,from,zipcode,income,goal,date,go_out,career,career_c,sports,tvsports,exercise,dining,museums,art,hiking,gaming,clubbing,reading,tv,theater,movies,concerts,music,shopping,yoga,exphappy,expnum,attr1_1,sinc1_1,intel1_1,fun1_1,amb1_1,shar1_1,attr4_1,sinc4_1,intel4_1,fun4_1,amb4_1,shar4_1,attr2_1,sinc2_1,intel2_1,fun2_1,amb2_1,shar2_1,attr3_1,sinc3_1,fun3_1,intel3_1,amb3_1,attr5_1,sinc5_1,intel5_1,fun5_1,amb5_1,dec,attr,sinc,intel,fun,amb,shar,like,prob,met,match_es,attr1_s,sinc1_s,intel1_s,fun1_s,amb1_s,shar1_s,attr3_s,sinc3_s,intel3_s,fun3_s,amb3_s,satis_2,length,numdat_2,attr7_2,sinc7_2,intel7_2,fun7_2,amb7_2,shar7_2,attr1_2,sinc1_2,intel1_2,fun1_2,amb1_2,shar1_2,attr4_2,sinc4_2,intel4_2,fun4_2,amb4_2,shar4_2,attr2_2,sinc2_2,intel2_2,fun2_2,amb2_2,shar2_2,attr3_2,sinc3_2,intel3_2,fun3_2,amb3_2,attr5_2,sinc5_2,intel5_2,fun5_2,amb5_2,you_call,them_cal,date_3,numdat_3,num_in_3,attr1_3,sinc1_3,intel1_3,fun1_3,amb1_3,shar1_3,attr7_3,sinc7_3,intel7_3,fun7_3,amb7_3,shar7_3,attr4_3,sinc4_3,intel4_3,fun4_3,amb4_3,shar4_3,attr2_3,sinc2_3,intel2_3,fun2_3,amb2_3,shar2_3,attr3_3,sinc3_3,intel3_3,fun3_3,amb3_3,attr5_3,sinc5_3,intel5_3,fun5_3,amb5_3,scored_decision,stdzed_scored_decision
2908,209,16.0,0,31,2,9,20,20,20.0,16,1,214.0,0,0.13,0,23.0,2.0,17.02,21.28,17.02,21.28,14.89,8.51,0,7.0,8.0,8.0,7.0,8.0,6.0,7.0,6.0,2.0,28.0,International Affairs/Business,13.0,NaN,NaN,NaN,1.0,7.0,3.0,New York,"12,601","30,855.00",3.0,6.0,2.0,Country Analysis/Research/Credit Analysis,7.0,3.0,2.0,6.0,8.0,5.0,5.0,6.0,8.0,8.0,3.0,6.0,7.0,8.0,7.0,10.0,3.0,3.0,7.0,NaN,13.04,21.74,17.39,17.39,15.22,15.22,7.0,7.0,8.0,7.0,8.0,8.0,20.93,13.95,13.95,18.6,13.95,18.6,7.0,9.0,8.0,7.0,8.0,NaN,NaN,NaN,NaN,NaN,0,6.0,5.0,5.0,6.0,5.0,5.0,NaN,NaN,NaN,0.0,13.04,19.57,15.22,19.57,15.22,17.39,7.0,9.0,7.0,9.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2909,209,16.0,0,31,2,9,20,20,20.0,18,2,215.0,0,-0.13,0,26.0,4.0,14.89,19.15,21.28,19.15,14.89,10.64,0,4.0,5.0,6.0,5.0,8.0,5.0,5.0,4.0,2.0,28.0,International Affairs/Business,13.0,NaN,NaN,NaN,1.0,7.0,3.0,New York,"12,601","30,855.00",3.0,6.0,2.0,Country Analysis/Research/Credit Analysis,7.0,3.0,2.0,6.0,8.0,5.0,5.0,6.0,8.0,8.0,3.0,6.0,7.0,8.0,7.0,10.0,3.0,3.0,7.0,NaN,13.04,21.74,17.39,17.39,15.22,15.22,7.0,7.0,8.0,7.0,8.0,8.0,20.93,13.95,13.95,18.6,13.95,18.6,7.0,9.0,8.0,7.0,8.0,NaN,NaN,NaN,NaN,NaN,0,8.0,NaN,8.0,NaN,8.0,NaN,NaN,NaN,NaN,0.0,13.04,19.57,15.22,19.57,15.22,17.39,7.0,9.0,7.0,9.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2910,209,16.0,0,31,2,9,20,20,20.0,5,3,216.0,0,0.09,0,27.0,2.0,7.50,17.50,20.00,22.50,12.50,20.00,1,6.0,9.0,9.0,8.0,7.0,6.0,6.0,5.0,2.0,28.0,International Affairs/Business,13.0,NaN,NaN,NaN,1.0,7.0,3.0,New York,"12,601","30,855.00",3.0,6.0,2.0,Country Analysis/Research/Credit Analysis,7.0,3.0,2.0,6.0,8.0,5.0,5.0,6.0,8.0,8.0,3.0,6.0,7.0,8.0,7.0,10.0,3.0,3.0,7.0,NaN,13.04,21.74,17.39,17.39,15.22,15.22,7.0,7.0,8.0,7.0,8.0,8.0,20.93,13.95,13.95,18.6,13.95,18.6,7.0,9.0,8.0,7.0,8.0,NaN,NaN,NaN,NaN,NaN,0,5.0,6.0,7.0,8.0,NaN,NaN,NaN,5.0,2.0,0.0,13.04,19.57,15.22,19.57,15.22,17.39,7.0,9.0,7.0,9.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [63]:
preferences_0 = wave9[wave9['gender']==0].pivot(index='iid', columns='pid', values='stdzed_scored_decision').fillna(0)
preferences_1 = wave9[wave9['gender']==1].pivot(index='iid', columns='pid', values='stdzed_scored_decision').fillna(0)

In [72]:
preference_table = preferences_0 + preferences_1.T

In [ ]:
preference_table

In [64]:
preferences_0.to_csv('preferences_0.csv')
preferences_1.to_csv('preferences_1.csv')

In [ ]:
pref

In [46]:
matched_partners = wave9[wave9['gender']==0].merge(wave9[wave9['gender']==1], left_on=['pid', 'iid'], right_on=['iid', 'pid'], suffixes=('_0', '_1'))

,iid_0,gender_0,pid_0,stdzed_scored_decision_0,iid_1,gender_1,pid_1,stdzed_scored_decision_1
0,194,0,214.0,0.700783,214,1,194.0,-1.769323
1,194,0,215.0,1.605020,215,1,194.0,-1.758306
2,194,0,216.0,-1.559808,216,1,194.0,-2.340994
3,194,0,217.0,-0.203453,217,1,194.0,-0.420684
4,194,0,218.0,-0.203453,218,1,194.0,-1.133980


In [53]:
matched_partners.to_csv('matched_partners_wave_9.csv')

In [114]:
df['date_only'] = df['timestamp'].str.split(' ').str[0]

In [118]:
oneday = df[df['date_only']=='2019-11-14']

In [123]:
#bag of words 
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
total_cars_matrix = cv.fit_transform(oneday['cars_list'])

In [127]:
#turn into dataframe

total_cars_df = pd.DataFrame(total_cars_matrix.toarray(), columns=cv.get_feature_names_out())

In [145]:
combodf = pd.concat([oneday.reset_index(drop=True), total_cars_df], axis=1)

In [151]:
combodf

,latitude,longitude,total_cars,cars_list,timestamp,date_only,10,100,101,102,...,90,91,92,93,94,95,96,97,98,99
0,32.091010,34.786330,1,[197],2019-11-14 00:03:03 UTC,2019-11-14,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,32.124610,34.830230,1,[116],2019-11-14 00:03:03 UTC,2019-11-14,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,32.097650,34.800800,1,[103],2019-11-14 00:03:03 UTC,2019-11-14,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,32.078621,34.797969,1,[203],2019-11-14 00:03:03 UTC,2019-11-14,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,32.076318,34.767295,1,[248],2019-11-14 00:03:03 UTC,2019-11-14,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160353,32.079825,34.774450,2,"[148, 67]",2019-11-14 21:33:02 UTC,2019-11-14,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
160354,32.143248,34.792340,3,"[94, 139, 272]",2019-11-14 21:33:02 UTC,2019-11-14,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
160355,32.064615,34.795787,3,"[259, 72, 153]",2019-11-14 21:33:02 UTC,2019-11-14,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
160356,32.108610,34.797300,1,[55],2019-11-14 21:33:02 UTC,2019-11-14,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [166]:
groupedup = oneday.sort_values('timestamp').groupby(['latitude', 'longitude', 'timestamp'])[['total_cars']].mean()

In [174]:
groupedup['rolling'] = groupedup['total_cars'].rolling(2).mean()

In [190]:
grouped = oneday.sort_values('timestamp').groupby(['latitude', 'longitude']).rolling(2, on='timestamp')[['total_cars']].mean()


In [202]:
grouped['cars'] = oneday.sort_values('timestamp').groupby(['latitude', 'longitude', 'timestamp'])[['total_cars']].mean()

In [203]:
grouped[(grouped['total_cars'] != grouped['cars']) & (grouped['total_cars'].notnull())]

total_cars  cars
latitude  longitude timestamp                                
32.034470 34.74685  2019-11-14 14:39:03 UTC         1.5   1.0
32.035393 34.75873  2019-11-14 04:09:02 UTC         0.5   0.0
                    2019-11-14 12:30:03 UTC         0.5   1.0
                    2019-11-14 12:36:02 UTC         0.5   0.0
32.036900 34.76250  2019-11-14 05:03:02 UTC         0.5   1.0
...                                                 ...   ...
32.143248 34.79234  2019-11-14 11:06:02 UTC         3.5   3.0
                    2019-11-14 13:03:02 UTC         3.5   4.0
                    2019-11-14 16:54:09 UTC         3.5   3.0
                    2019-11-14 18:18:03 UTC         3.5   4.0
                    2019-11-14 19:54:02 UTC         3.5   3.0

[1982 rows x 2 columns]

In [225]:
grouped.loc[(grouped['total_cars'] - grouped['cars']) > 0, 'Demand'] = 1

grouped.loc[(grouped['total_cars'] - grouped['cars']) < 0, 'Supply'] = 1

In [227]:
grouped.Demand.sum()

997.0

In [228]:
grouped.Supply.sum()

985.0

In [220]:
oneday[oneday['latitude']==32.072323].sort_values('timestamp')[250:300]

,latitude,longitude,total_cars,cars_list,timestamp,date_only
7119225,32.072323,34.790555,2,"[18, 54]",2019-11-14 12:30:03 UTC,2019-11-14
7082557,32.072323,34.790555,2,"[18, 54]",2019-11-14 12:33:03 UTC,2019-11-14
7085497,32.072323,34.790555,2,"[18, 54]",2019-11-14 12:36:02 UTC,2019-11-14
7082993,32.072323,34.790555,2,"[18, 54]",2019-11-14 12:39:02 UTC,2019-11-14
7092512,32.072323,34.790555,2,"[18, 54]",2019-11-14 12:42:02 UTC,2019-11-14
7115525,32.072323,34.790555,2,"[18, 54]",2019-11-14 12:45:02 UTC,2019-11-14
7026960,32.072323,34.790555,2,"[18, 54]",2019-11-14 12:48:03 UTC,2019-11-14
7118317,32.072323,34.790555,2,"[18, 54]",2019-11-14 12:51:02 UTC,2019-11-14
7124652,32.072323,34.790555,2,"[18, 54]",2019-11-14 12:54:03 UTC,2019-11-14
7122201,32.072323,34.790555,2,"[18, 54]",2019-11-14 12:57:03 UTC,2019-11-14


In [99]:
# turn into a dataframe
total_cars_matrix[:,4].sum()

0

In [86]:
total_cars_df.sum()

0       0
1       0
2       0
3       0
4       0
       ..
256     7
257    26
258    29
259    29
260    27
Length: 261, dtype: int64

In [35]:
# make cars_list a list

df['cars_list'].str.sl

0           [[138]]
1            [[64]]
2           [[120]]
3           [[110]]
4           [[190]]
             ...   
20049193       [[]]
20049194       [[]]
20049195       [[]]
20049196       [[]]
20049197       [[]]
Name: cars_list, Length: 20049198, dtype: object

In [33]:
df[df['cars_list'].apply(lambda x: 71 in x)]

TypeError: 'in <string>' requires string as left operand, not int